## Final Project name
Tytus Suchotinunt
DS 4003

### Data Introduction: What Is All This Data?
I have chosen 3 datasets, which were found in the links listed below.

https://transitapp.com/apta

https://www.apta.com/research-technical-resources/transit-statistics/ridership-report/ridership-report-archives/

This data was from the American Public Transportation Association (APTA), a nonprofit international association of more than 1,500 public and private transportation sector member organizations. The main idea of all 3 datasets is to track, monitor, and analyze public transportation ridership trends throughout America over the years. The different datasets vary in the scope of data collected, with the biggest dataset tracking up-to-date ridership estimates each week for every partner organization. The other two datasets are more general, simply showing aggregate ridership numbers over the years, broken down by different modes of public transportation. 

While these ridership numbers do not account for every rider in America, they do a good job of showing pretty accurate trends within the industry. APTA is, afterall, a very trusted organization within the public transportation sector. These ridership values are modeled based on measures of Transit app usage (usage of these public transportation's app) to provide a current measure of demand for public transit. While these estimates do not represent actual reported ridership counts from agencies, however it was found that app’s user demographics generally match those of public transit riders overall, both compared to other common survey methods and available U.S. Census data.

## Data Cleaning

In [2]:
# import all libraries
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px

In [3]:
# load in dataset
upt_prediction = pd.read_csv('upt_prediction.csv')
mode_quarter = pd.read_csv('APTA-mode-and-quarter.csv')
year_end = pd.read_csv('APTA-year-end-totals.csv')

#Dataset head if need to see how it loads in
#upt_prediction.head()
#mode_quarter.head()
#year_end.head()

In [4]:
# analyze the upt_prediction dataset
print(upt_prediction.head())
print(upt_prediction.dtypes)


    Week of  Prediction                          Name  \
0  2/4/2018    248188.0              Foothill Transit   
1  2/4/2018    109761.0                      Unitrans   
2  2/4/2018    827602.0     Orange County Transp Auth   
3  2/4/2018     28367.0     San Joaquin Reg Rail Comm   
4  2/4/2018    112487.0  Tompkins Consol Area Transit   

                      APTA Name        Size        Region         City  \
0              Foothill Transit         2M+       Pacific  Los Angeles   
1                      Unitrans  Under 500k       Pacific   Sacramento   
2     Orange County Transp Auth         2M+       Pacific  Los Angeles   
3     San Joaquin Reg Rail Comm  Under 500k       Pacific  SF Bay Area   
4  Tompkins Consol Area Transit  Under 500k  Mid-Atlantic       Ithaca   

    NTD ID  Feed ID  
0  90146.0    173.0  
1  90142.0    774.0  
2  90036.0     86.0  
3  90182.0    642.0  
4  20145.0    619.0  
Week of        object
Prediction    float64
Name           object
APTA Name     

In [5]:
# drop the columns that I will not use
upt_drop = ['NTD ID', 'Feed ID']
upt_prediction.drop(upt_drop, inplace=True, axis=1)

# drop all rows with unknown/missing values
upt_prediction.dropna()

# change week type from obj to datetime
upt_prediction['Week of'] = pd.to_datetime(upt_prediction['Week of'])
print(upt_prediction.dtypes)

# Make sure the dataset looks good!
upt_prediction.head()


Week of       datetime64[ns]
Prediction           float64
Name                  object
APTA Name             object
Size                  object
Region                object
City                  object
dtype: object


,Week of,Prediction,Name,APTA Name,Size,Region,City
0,2018-02-04,248188.0,Foothill Transit,Foothill Transit,2M+,Pacific,Los Angeles
1,2018-02-04,109761.0,Unitrans,Unitrans,Under 500k,Pacific,Sacramento
2,2018-02-04,827602.0,Orange County Transp Auth,Orange County Transp Auth,2M+,Pacific,Los Angeles
3,2018-02-04,28367.0,San Joaquin Reg Rail Comm,San Joaquin Reg Rail Comm,Under 500k,Pacific,SF Bay Area
4,2018-02-04,112487.0,Tompkins Consol Area Transit,Tompkins Consol Area Transit,Under 500k,Mid-Atlantic,Ithaca


In [6]:
# analyze the mode_quarter dataset
print(mode_quarter.head())
print(mode_quarter.dtypes)

  Quarter Unnamed: 1  Year Total Ridership Heavy Rail Light Rail  \
0      Q1  1990 - Q1  1990       2,256,422    606,580     36,898   
1      Q2  1990 - Q2  1990       2,163,608    577,237     40,470   
2      Q3  1990 - Q3  1990       2,071,195    546,929     41,676   
3      Q4  1990 - Q4  1990       2,198,460    567,572     43,093   
4      Q1  1991 - Q1  1991       2,068,031    515,379     39,260   

  Commuter Rail Trolleybus        Bus Demand Response Other  
0        82,274        NaN  1,457,932          18,223   NaN  
1        84,150        NaN  1,389,602          17,703   NaN  
2        81,481        NaN  1,325,863          16,614   NaN  
3        82,754        NaN  1,435,452          17,514   NaN  
4        79,746        NaN  1,364,428          17,808   NaN  
Quarter            object
Unnamed: 1         object
Year                int64
Total Ridership    object
Heavy Rail         object
Light Rail         object
Commuter Rail      object
Trolleybus         object
Bus        

In [7]:
# drop the columns that I will not use
mode_quarter.drop('Unnamed: 1', inplace=True, axis=1)

# trying to change all columns from obj to float, except the Quarter and year column, so single it out
columns_to_convert1 = mode_quarter.columns[~mode_quarter.columns.isin(['Quarter', 'Year'])]

# in order to change all obj to float, first remove the commas from the numbers
mode_quarter[columns_to_convert1] = mode_quarter[columns_to_convert1].applymap(lambda x: float(str(x).replace(',', '')))

# change all columns from obj to float (except Quarter column)
mode_quarter[columns_to_convert1] = mode_quarter[columns_to_convert1].astype(float)

# Make sure the dataset looks good!
print(mode_quarter.head())
print(mode_quarter.dtypes)

  Quarter  Year  Total Ridership  Heavy Rail  Light Rail  Commuter Rail  \
0      Q1  1990        2256422.0    606580.0     36898.0        82274.0   
1      Q2  1990        2163608.0    577237.0     40470.0        84150.0   
2      Q3  1990        2071195.0    546929.0     41676.0        81481.0   
3      Q4  1990        2198460.0    567572.0     43093.0        82754.0   
4      Q1  1991        2068031.0    515379.0     39260.0        79746.0   

   Trolleybus        Bus  Demand Response  Other  
0         NaN  1457932.0          18223.0    NaN  
1         NaN  1389602.0          17703.0    NaN  
2         NaN  1325863.0          16614.0    NaN  
3         NaN  1435452.0          17514.0    NaN  
4         NaN  1364428.0          17808.0    NaN  
Quarter             object
Year                 int64
Total Ridership    float64
Heavy Rail         float64
Light Rail         float64
Commuter Rail      float64
Trolleybus         float64
Bus                float64
Demand Response    float64


In [8]:
# analyze the year_end dataset
print(year_end.head())
print(year_end.dtypes)

     Year Total Ridership Heavy Rail Light Rail Commuter Rail Trolleybus  \
0  1990.0       8,956,479  2,420,196    146,443       327,547   in other   
1  1991.0       8,483,877  2,182,759    170,726       325,186   in other   
2  1992.0       8,555,107  2,064,773    162,994       326,443   in other   
3  1993.0       8,452,120  2,172,905    168,451       329,591    125,464   
4  1994.0       8,450,736  2,278,945    232,884       349,542    118,524   

         Bus Demand Response    Other  
0  5,740,648         106,984  214,660  
1  5,526,171          72,108  206,927  
2  5,699,502          98,116  203,280  
3  5,487,952          94,119   73,639  
4  5,284,821          98,024   87,994  
Year               float64
Total Ridership     object
Heavy Rail          object
Light Rail          object
Commuter Rail       object
Trolleybus          object
Bus                 object
Demand Response     object
Other               object
dtype: object


In [9]:
# change year from float to int
year_end['Year'] = year_end['Year'].astype('Int64')

# replace 'in other' value in 'Trolley' column with NaN
year_end['Trolleybus'].replace('in other', np.nan, inplace=True)

# trying to change all columns from obj to float, except the year column, so single it out
columns_to_convert2 = year_end.columns[year_end.columns != 'Year']

# in order to change all obj to float, first remove the commas from the numbers
year_end[columns_to_convert2] = year_end[columns_to_convert2].applymap(lambda x: float(str(x).replace(',', '')))

# change all columns from obj to float (except year column)
year_end[columns_to_convert2] = year_end[columns_to_convert2].astype(float)

# Make sure the dataset looks good!
print(year_end.head())
print(year_end.dtypes)

   Year  Total Ridership  Heavy Rail  Light Rail  Commuter Rail  Trolleybus  \
0  1990        8956479.0   2420196.0    146443.0       327547.0         NaN   
1  1991        8483877.0   2182759.0    170726.0       325186.0         NaN   
2  1992        8555107.0   2064773.0    162994.0       326443.0         NaN   
3  1993        8452120.0   2172905.0    168451.0       329591.0    125464.0   
4  1994        8450736.0   2278945.0    232884.0       349542.0    118524.0   

         Bus  Demand Response     Other  
0  5740648.0         106984.0  214660.0  
1  5526171.0          72108.0  206927.0  
2  5699502.0          98116.0  203280.0  
3  5487952.0          94119.0   73639.0  
4  5284821.0          98024.0   87994.0  
Year                 Int64
Total Ridership    float64
Heavy Rail         float64
Light Rail         float64
Commuter Rail      float64
Trolleybus         float64
Bus                float64
Demand Response    float64
Other              float64
dtype: object


In [9]:
# exporting all my data to 3 seperate csv files

upt_prediction.to_csv('data1.csv', index=False)
mode_quarter.to_csv('data2.csv', index=False)
year_end.to_csv('data3.csv', index=False)

## Exploratory Analysis
Data1 (the upt_prediction csv) has a total of 41580 rows of data, which is a lot of observations! Unlike the other two datasets, this one also has many unique categories. There are 132 unique transportation agency's data within this dataset, spanning across 69 cities and 8 regions of the United States. Within these transportation systems, there is also 4 unique sizes of transportation services (from small organizations to organizations that serve over 2 million people). There were very few Null values in the dataset. I ended up dropping them due to there being minimal null values compared to the amount of observations, so it'd be best just to clean my dataset.

Data2 (the mode_quarter csv) has a total of 135 observations, which spans over 33 years of collected data, each broken up quarterly. This dataset did have missing ridership values within the first few years of the trolleybus column, I decided to drop these values. That data, according to the website, was left blank because those first few years the trolleybus ridership data was instead put into the "other" column. This is something to keep in mind when looking at graphs produced by this data later down the road. Looking at the basic stastical analysis of this dataset shows that there are clear favorites of public transportation method, that being heavy rail and bus.

Data3 (the year_end csv) has a total of 33 observations, one for each year these past 33 years. This dataset also had missing values for the first few years of the trolleybus column. The reasoing was the same as cited above, so it is important to keep this in mind when doing data analysis. As stated above, looking at the preliminary statstical analysis shows that the most used public transportation methods would be heavy rail and bus. From viewing the ridership numbers, it is clear that the covid-19 pandemic has negatively impacted ridership numbers. While ridership is on the rise post pandemic, it still has not reached pre-pandemic levels. It would be interesting to see this visualized!

In [12]:
# have a quick overview of the dataset
print(upt_prediction.info())
print(mode_quarter.info())
print(year_end.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41580 entries, 0 to 41579
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Week of     41580 non-null  datetime64[ns]
 1   Prediction  41580 non-null  float64       
 2   Name        41580 non-null  object        
 3   APTA Name   41580 non-null  object        
 4   Size        41580 non-null  object        
 5   Region      41580 non-null  object        
 6   City        41265 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 2.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Quarter          135 non-null    object 
 1   Year             135 non-null    int64  
 2   Total Ridership  135 non-null    float64
 3   Heavy Rail       135 non-null    float64
 4 

In [17]:
# Quick statstical summary of data2 and data3 datasets
print(mode_quarter.describe())
print(year_end.describe())

              Year  Total Ridership    Heavy Rail     Light Rail  \
count   135.000000     1.350000e+02  1.350000e+02     135.000000   
mean   2006.377778     2.271985e+06  7.202221e+05   89929.125926   
std       9.779560     3.989737e+05  1.894795e+05   32088.489250   
min    1990.000000     6.209250e+05  1.222490e+05   33137.000000   
25%    1998.000000     2.077336e+06  5.717830e+05   65349.000000   
50%    2006.000000     2.366098e+06  6.781960e+05   83869.000000   
75%    2015.000000     2.563128e+06  9.051295e+05  123489.000000   
max    2023.000000     2.729424e+06  1.015234e+06  140828.000000   

       Commuter Rail    Trolleybus           Bus  Demand Response  \
count     135.000000    127.000000  1.350000e+02       135.000000   
mean    99984.688889  24594.834646  1.267697e+06     36109.170370   
std     22460.322259   6467.580604  2.181805e+05     13370.207006   
min     13521.000000   3395.000000  4.205780e+05     16614.000000   
25%     87739.000000  21853.500000  1.2429

In [16]:
# unique values in upt_pred
upt_prediction.nunique()

Week of         315
Prediction    39841
Name            132
APTA Name       132
Size              4
Region            8
City             69
dtype: int64

## Data Dictionary

### Dataset 1
Week of - The time of which the data was collected, in week format using M/D/Y.

Prediction - The (pretty accurate) estimated amount of riders of public transportation

Name - The name of the public transportation provider

APTA Name - The name of the public transportation provider within the American Public Transportation Association system (the actual names vs the names within the APTA system simply may differ)

Size - The size of the specified public transportation system based on the amount of people they serve

Region - The US region in which the public transportation system is located and serves

City - The city in which the public transportation system is located and serves

### Dataset 2 and Dataset 3 (they share many of the same variables)
Quarter - The quarter of the year in which the data was collected (Q1 being Jan-March, Q2 being April-June, Q3 being July-September, Q4 being October-December)

Year - The year in which the data was collected

Total Ridership - The total ridership documented for all public transportation modes

Heavy Rail - The total ridership for heavy rail, a mode of transit service (also called metro, subway, rapid transit, or rapid rail) operating on an electric railway with the capacity for a heavy volume of traffic. It is characterized by high speed and rapid acceleration passenger rail cars operating singly or in multi‐car trains on fixed rails; separate rights‐of‐way from which all other vehicular and foot traffic are excluded; sophisticated
signaling, and high platform loading

Light Rail - The total ridership from light rail, a mode of transit service (also called streetcar, tramway, or trolley) operating passenger rail cars singly (or in short, usually two‐car or three‐car, trains) on fixed rails in right‐of‐way that is often separated from other traffic for part or much of the way. Light rail vehicles are typically driven electrically with power being drawn from an overhead electric line via a trolley or a pantograph; driven by an operator on board the vehicle; and may have either high platform loading or low level boarding using steps

Commuter Rail - The total ridership from commuter rail, a mode of transit service (also called metropolitan rail, regional rail, or suburban
rail) characterized by an electric or diesel propelled railway for urban passenger train service consisting of local short distance travel operating between a central city and adjacent suburbs. Such rail service, using either locomotive hauled or self‐propelled railroad passenger cars, is generally characterized by multi‐trip tickets, specific station to station fares, railroad employment practices and usually only one or two stations in the central business district. Intercity rail service is excluded, except for that portion of such service that is operated by or under contract with a public transit agency for predominantly commuter services. Most service is provided on routes of current or former freight railroads

Trolleybus - The total ridership from trolleybus, a mode of transit service (also called trolley coach) using vehicles propelled by a motor drawing current from overhead wires via connecting poles called a trolley poles from a central power source not on board the vehicle

Bus - The total ridership from bus, a mode of transit service (also called motor bus) characterized by roadway vehicles powered by diesel, gasoline, battery, or alternative fuel engines contained within the vehicle. Vehicles operate on streets and roadways in fixed‐route or other regular service. Types of bus service include local service, where vehicles may stop every block or two along a route several miles long

Demand Response - The total ridership from demand response, a mode of transit service (also called Paratransit or dial‐a‐ride) characterized by the use of passenger automobiles, vans or small buses operating in response to calls from passengers or their agents to the transit operator, who then dispatches a vehicle to pick up the passengers and transport them to their destinations. The vehicles do not operate over a fixed route or on a fixed schedule. The vehicle may be dispatched to pick up several passengers at different pick‐up points before taking them to their respective destinations and may even be interrupted en route to these destinations to pick up other passengers 

Other - The total ridership from other sources of public transportation. This section can include ferry boat trips, cable cars, etc

## Brainstorming

Some possible UI components for this project could be a slider to progress through the time periods, a dropdown menu to provide options for filtering the data (looking at only heavy rail, or looking at only east coast systems), a text input to help select from the list of 50 avaiable cities, a button that would then trigger some type of update to the dashboard, and of course charts and graphs to visualize all this data.

For data visualizaion, I see a line graph fitting very nicely due to the time x axis against a y ridership number axis. With this visualization, different colored lines can represent different filters (such as mode of transport or region). I also would find a pie chart intersting to see the distribution of different modes of transportation. I would also find it interesting to do a map to plot locations of transportation agencies and use sizes of said points to represent total ridership. Similarily, I could do a heatmap to show the distribution of ridership across America.